In [1]:
!pip install num2words
%pip install -q ipdb
%env PYTHONBREAKPOINT=IPython.core.debugger.set_trace
%pdb on

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 13.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=114c012cbb870ca1b2ab91da8f311deb48393a2c0792eeaf1b86e1024a3df5c7
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.0 MB/s eta 0:00:00
env: PYTHONBREAKPOINT=IPython.core.debugger.set_trace
Automatic pdb calling has been turned ON


In [2]:
!pip3 install lerobot==0.3.3

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.7/597.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 914.2 kB/s et

In [109]:
def set_deterministic(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False
    torch.set_num_threads(1)  # reduce CPU reduction-order noise

In [110]:
import torch
import random
import numpy as np
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig
from lerobot.configs.policies import PreTrainedConfig

from lerobot.datasets.lerobot_dataset import LeRobotDataset
from lerobot.datasets.lerobot_dataset import LeRobotDatasetMetadata

device = "cuda" if torch.cuda.is_available() else "cpu"

config = PreTrainedConfig.from_pretrained("lerobot/smolvla_base")
ds = LeRobotDataset("lerobot/svla_so101_pickplace")
ds_meta = LeRobotDatasetMetadata("lerobot/svla_so101_pickplace")
policy = SmolVLAPolicy(config, dataset_stats=ds_meta.stats).to(device).eval()

set_deterministic(0)

ds = LeRobotDataset("lerobot/svla_so101_pickplace")

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading  HuggingFaceTB/SmolVLM2-500M-Video-Instruct weights ...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Reducing the number of VLM layers to 16 ...


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [111]:
from lerobot.envs.utils import preprocess_observation
import numpy as np
import ipdb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


sample = ds[0]

batch = {
    "observation.image":   sample["observation.images.up"].unsqueeze(0).to(device),   # [B,C,H,W]
    "observation.image2": sample["observation.images.side"].unsqueeze(0).to(device), # [B,C,H,W]
    "observation.state":       sample["observation.state"].unsqueeze(0).to(device),       # [B,6]
    "task": ["Pick and place the object"],  # dataset is single-task; any reasonable string works
}



In [112]:
# ipdb.set_trace()

policy.reset()
# print(batch)
with torch.inference_mode():
    torch.cuda.synchronize()
    action = policy.select_action(batch)
    torch.cuda.synchronize()
print(action)

tensor([[-16.5644, -65.1996, -21.6938,  76.2062, -40.3970,   0.6886]],
       device='cuda:0')


tensor([[-16.5644, -65.1996, -21.6938,  76.2062, -40.3970,   0.6886]],
       device='cuda:0')

b /usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/modeling_smolvla.py:893

b /usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/modeling_smolvla.py:807